## Anchor explanations on Adult dataset

In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular
import pickle

### Load the train and test set

In [ ]:
nome = 'adult'
title = "../datasets/train_set_"+nome+"_strat.p"
train = open(title,"rb")
train_set = pickle.load(train)
title = "../datasets/train_label_"+nome+"_strat.p"
train_l = open(title,"rb")
train_label = pickle.load(train_l)
title = "../datasets/test_set_" + nome + "_strat.p"
test = open(title, "rb")
test_set = pickle.load(test)
title = "../datasets/test_label_" + nome + "_strat.p"
test_l = open(title, "rb")
test_label = pickle.load(test_l)

### Define and fit the model 

In [ ]:
from xgboost import XGBClassifier
bb = XGBClassifier(C= 1, penalty='l2', objective='binary:logistic', seed = 42, bootstrap=True, max_depth=90,learning_rate=0.1, n_estimators=500, tree_method='auto')
bb.fit(train_set.values, train_label.values)

### Define the Anchor tabular explainer
#### We pass the train set, the columns and the unique values of the labels 

In [ ]:
explainer = anchor_tabular.AnchorTabularExplainer(
    class_names=train_label.unique,
    feature_names=train_set.columns,
    train_data=train_set.values
)

### Select one record from the test set
#### We query the model to get the prediction: in thi case it is part of class =< 50k

In [ ]:
idx = 0
print(train_set.iloc[idx])
bb.predict(train_set.iloc[idx].values.reshape(1, -1))


### We explain the record selected
#### We set the parameter threshold 

In [ ]:
exp = explainer.explain_instance(
    data_row=train_set.iloc[idx].values.reshape(1, -1),
    classifier_fn=bb.predict,
    threshold=0.95
)

In [ ]:
print('Anchor explanation: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())


### We select another record, with a different prediction

In [ ]:
idx = 23
print(train_set.iloc[idx].values)
bb.predict(train_set.iloc[idx].values.reshape(1, -1))

In [ ]:
exp = explainer.explain_instance(test_set.iloc[idx].values.reshape(1, -1), bb.predict, threshold=0.95)

In [ ]:
print('Anchor explanation: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())